# Testing cross project artifacts

In [1]:
!pip install -U xgboost

Requirement already up-to-date: xgboost in /User/.pythonlibs/jupyter/lib/python3.7/site-packages (1.6.2)


## 1. artifact generating function 

In [2]:
#mlrun: start-code

import mlrun
import pandas as pd 
import json
import os
from xgboost import XGBClassifier
import pickle
from mlrun.artifacts.base import DirArtifact
from mlrun import MLClientCtx
from sklearn.datasets import load_iris
from io import BytesIO
from sklearn.model_selection import train_test_split

def get_dataitem(context: MLClientCtx,
                             key: str):
    
    for artifact in context.artifacts:
        if artifact['kind'] == 'model' and artifact['metadata'].get('key',None) == key:
            return mlrun.get_dataitem(artifact['spec']['target_path'] + artifact['spec']['model_file'])
        elif artifact['kind'] == 'dataset' and artifact['metadata'].get('key',None) == key:
            return mlrun.get_dataitem(artifact['spec']['target_path'])
        elif artifact['metadata'].get('key',None) == key:
            return mlrun.get_dataitem(artifact['spec']['target_path'])
    context.logger.info('Artifact not found')
    
def log_transactions(context: MLClientCtx,
                    ):
        
    # uploading new artifact 
    df_encode = pd.DataFrame(load_iris()['data']).to_json().encode()
    context.log_artifact('encoded_iris-'+context.artifact_path[:2], body=df_encode, local_path='encoded_iris-'+context.artifact_path[:2]+'.csv')
    # reading artifact
    trans_df = pd.DataFrame(json.loads(get_dataitem(context, 'encoded_iris-'+context.artifact_path[:2]).get()))
    context.logger.info(f'dataframe shape : {trans_df.shape}')
    
    # training the model (for serving purposes )
    bst = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')
    X,y = load_iris(return_X_y=True)
    X_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)
    bst.fit(X_train, y_train)
    # logging a model
    context.log_model('bst_model', body=pickle.dumps(bst), model_file='bst.pkl')
    # getting the model remote
    model = pickle.loads(get_dataitem(context, 'bst_model').get())
    context.logger.info(f'logged model : {model.__class__}')
    
    # Logging directory
    context.log_artifact(DirArtifact(key='my_project', target_path=context.artifact_path))
    
    # Logging dataset 
    context.log_dataset(key = 'iris_dataset-'+context.artifact_path[:2],
                        df = pd.DataFrame(json.loads(get_dataitem(context, 'encoded_iris-'+context.artifact_path[:2]).get())),
                        local_path='iris_dataset-'+context.artifact_path[:2]+'.csv')
    # Getting dataset
    context.logger.info(f'logged dataset {get_dataitem(context, "iris_dataset-" + context.artifact_path[:2]).as_df().shape}')
    
    return
            
#mlrun: end-code

## 2. Creating projects, setting & running artifact generating function

In [3]:
import mlrun
import os

# Initialize the MLRun project object
project1 = mlrun.get_or_create_project('cross-project1',user_project=True,context=os.path.join(os.getcwd(), 'test-notebooks1'))

project1.set_function(name='log_transactions', kind='job', image='mlrun/ml-models', handler='log_transactions')

project1.get_function('log_transactions').run(local=False)

> 2023-01-11 12:59:12,470 [info] Created and saved project cross-project1-admin: {'from_template': None, 'overwrite': False, 'context': '/User/test-notebooks/project_transfer/test-notebooks1', 'save': True}
> 2023-01-11 12:59:12,471 [info] created project cross-project1 and saved in MLRun DB
> 2023-01-11 12:59:29,820 [info] starting run log-transactions-log_transactions uid=04f3db71541a45faab5fb4ef98896deb DB=http://mlrun-api:8080
> 2023-01-11 12:59:30,002 [info] Job is running in the background, pod: log-transactions-log-transactions-62lzt
> 2023-01-11 12:59:35,254 [info] dataframe shape : (150, 4)
> 2023-01-11 12:59:35,331 [info] logged model : <class 'xgboost.sklearn.XGBClassifier'>
> 2023-01-11 12:59:35,468 [info] logged dataset (150, 5)
> 2023-01-11 12:59:35,515 [info] To track results use the CLI: {'info_cmd': 'mlrun get run 04f3db71541a45faab5fb4ef98896deb -p cross-project1-admin', 'logs_cmd': 'mlrun logs 04f3db71541a45faab5fb4ef98896deb -p cross-project1-admin'}
> 2023-01-11 12

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
cross-project1-admin,...98896deb,0,Jan 11 12:59:34,completed,log-transactions-log_transactions,v3io_user=adminkind=jobowner=adminmlrun/client_version=1.3.0-rc4host=log-transactions-log-transactions-62lzt,,,,encoded_iris-v3bst_modelmy_projectiris_dataset-v3


> 2023-01-11 12:59:36,329 [info] run executed, status=completed


In [4]:
# Initialize the MLRun project object
project2 = mlrun.get_or_create_project('cross-project2',user_project=True,context=os.path.join(os.getcwd(), 'test-notebooks2'))

project2.set_function(name='log_transactions', kind='job', image='mlrun/ml-models', handler='log_transactions')

project2.get_function('log_transactions').run(local=False)

> 2023-01-11 12:59:51,666 [info] Created and saved project cross-project2-admin: {'from_template': None, 'overwrite': False, 'context': '/User/test-notebooks/project_transfer/test-notebooks2', 'save': True}
> 2023-01-11 12:59:51,667 [info] created project cross-project2 and saved in MLRun DB
> 2023-01-11 13:00:10,458 [info] starting run log-transactions-log_transactions uid=c55bb3b538324b5180de9e068ea916f5 DB=http://mlrun-api:8080
> 2023-01-11 13:00:10,688 [info] Job is running in the background, pod: log-transactions-log-transactions-dpbx2
> 2023-01-11 13:00:15,740 [info] dataframe shape : (150, 4)
> 2023-01-11 13:00:15,825 [info] logged model : <class 'xgboost.sklearn.XGBClassifier'>
> 2023-01-11 13:00:15,972 [info] logged dataset (150, 5)
> 2023-01-11 13:00:16,029 [info] To track results use the CLI: {'info_cmd': 'mlrun get run c55bb3b538324b5180de9e068ea916f5 -p cross-project2-admin', 'logs_cmd': 'mlrun logs c55bb3b538324b5180de9e068ea916f5 -p cross-project2-admin'}
> 2023-01-11 13

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
cross-project2-admin,...8ea916f5,0,Jan 11 13:00:15,completed,log-transactions-log_transactions,v3io_user=adminkind=jobowner=adminmlrun/client_version=1.3.0-rc4host=log-transactions-log-transactions-dpbx2,,,,encoded_iris-v3bst_modelmy_projectiris_dataset-v3


> 2023-01-11 13:00:16,957 [info] run executed, status=completed


## 3. Importing/Exporting artifacts

### 3.1 base artifact

In [5]:
# Exporting project2 artifact
project2.get_artifact('log-transactions-log_transactions_encoded_iris-v3').export('proj2_encoded_iris.yaml')

# Importing the artifact from project1
project1.import_artifact('proj2_encoded_iris.yaml',
                         new_key = 'imported_proj1_encoded_iris')

# Testing the imported artifact
pd.DataFrame(json.loads(project1.get_artifact('imported_proj1_encoded_iris').to_dataitem().get())).head()

,0,1,2,3
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


### 3.2 dataset artifact

In [6]:
# Exporting project2 model artifact
project2.get_artifact('log-transactions-log_transactions_iris_dataset-v3').export('proj2_dataset.yaml')

# importing model artifact from project2
project1.import_artifact('proj2_dataset.yaml',
                         new_key = 'imported_proj1_dataset')

# Testing the imported dataset artifact
project1.get_artifact('imported_proj1_dataset').to_dataitem().as_df()

,Unnamed: 0,0,1,2,3
0,0,5.1,3.5,1.4,0.2
1,1,4.9,3.0,1.4,0.2
2,2,4.7,3.2,1.3,0.2
3,3,4.6,3.1,1.5,0.2
4,4,5.0,3.6,1.4,0.2
...,...,...,...,...,...
145,145,6.7,3.0,5.2,2.3
146,146,6.3,2.5,5.0,1.9
147,147,6.5,3.0,5.2,2.0
148,148,6.2,3.4,5.4,2.3


### 3.3 model artifact

In [7]:
# Exporting project1 model artifact
project1.get_artifact('log-transactions-log_transactions_bst_model').export('proj1_model.yaml')

# importing model artifact from project2
project2.import_artifact('proj1_model.yaml',
                         new_key = 'imported_proj2_model')


# Testing the imported model artifact
project2_model = pickle.loads(project2.get_artifact('imported_proj2_model')._get_file_body())
project2_model.predict(pd.DataFrame(json.loads((project1.get_artifact('log-transactions-log_transactions_encoded_iris-v3').to_dataitem().get())))[:5])

array([0, 0, 0, 0, 0])

### 3.4 dir artifact

In [8]:
try: 
    # Exporting project2 dir artifact
    project1.get_artifact('log-transactions-log_transactions_my_project').export('proj1_dir.yaml')

    # importing dir artifact from project2
    project2.import_artifact('proj1_dir.yaml',
                             new_key = 'imported_proj2_dir')

    # Testing the imported dir artifact
    project2.get_artifact('imported_proj2_dir').to_dataitem().listdir()
except Exception as e:
    print(e)

## 4. Importing/Exporting functions

### 4.1 Serving function

#### 4.1.1 Creating serving function, adding model, predicting and deploying

In [9]:
# Setting project1 serving function
project1.set_function(name='proj1_serving',func = mlrun.new_function(name='proj1_serving', kind='serving',image='mlrun/ml-models', command = []))

# adding project2 model to project1 serving function
project1.get_function('proj1_serving').add_model(key = 'my_model',
                                                 class_name = "mlrun.frameworks.xgboost.XGBoostModelServer",
                                                 model_path = project2.get_artifact('log-transactions-log_transactions_bst_model').target_path)

In [10]:
# Testing the fused model serving function
server = project1.get_function('proj1_serving').to_mock_server()

server.test(body={'inputs': 
                  pd.DataFrame(json.loads(project2.get_artifact('log-transactions-log_transactions_encoded_iris-v3').to_dataitem().get())).values.tolist()[-5:]})

> 2023-01-11 13:00:18,042 [warning] run command, file or code were not specified
> 2023-01-11 13:00:27,584 [info] model my_model was loaded
> 2023-01-11 13:00:27,585 [info] Loaded ['my_model']


{'id': '2a09b2f3964e4fbd81c665a7fae549f7',
 'model_name': 'my_model',
 'outputs': [2, 2, 2, 2, 2]}

In [11]:
project1.deploy_function('proj1_serving')

> 2023-01-11 13:00:27,870 [info] Starting remote function deploy
2023-01-11 13:00:28  (info) Deploying function
2023-01-11 13:00:28  (info) Building
2023-01-11 13:00:28  (info) Staging files and preparing base images
2023-01-11 13:00:28  (info) Building processor image
2023-01-11 13:03:16  (info) Build complete
2023-01-11 13:03:34  (info) Function deploy complete
> 2023-01-11 13:03:34,977 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-cross-project1-admin-proj1-serving.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['cross-project1-admin-proj1-serving-cross-project1-admin.default-tenant.app.vmdev94.lab.iguazeng.com/']}


DeployStatus(state=ready, outputs={'endpoint': 'http://cross-project1-admin-proj1-serving-cross-project1-admin.default-tenant.app.vmdev94.lab.iguazeng.com/', 'name': 'cross-project1-admin-proj1-serving'})

In [12]:
import time
time.sleep(5)

project1.get_function('proj1_serving').invoke(
    path='/v2/models/my_model/infer', 
    body={'inputs': pd.DataFrame(json.loads(project2.get_artifact('log-transactions-log_transactions_encoded_iris-v3').to_dataitem().get())).values.tolist()[-5:]}
)

> 2023-01-11 13:03:40,132 [info] invoking function: {'method': 'POST', 'path': 'http://nuclio-cross-project1-admin-proj1-serving.default-tenant.svc.cluster.local:8080/v2/models/my_model/infer'}


{'id': 'b68735d6-5794-4391-9a1f-f3f4d4c38daa',
 'model_name': 'my_model',
 'outputs': [2, 2, 2, 2, 2]}

#### 4.1.2 Importing & Exporting already deployed serving function

In [13]:
# exporting project1 deployed serving function
project1.get_function('proj1_serving').export('proj1_serving.yaml')

# importing project1 serving function from project2
project2.set_function(name='proj2_imported_serving', func='../proj1_serving.yaml')
print(project2.get_function('proj2_imported_serving').is_deployed())

> 2023-01-11 13:03:40,442 [info] function spec saved to path: proj1_serving.yaml
True


In [14]:
try:
    # Testing the imported already deployed function
    server = project2.get_function('proj2_imported_serving').to_mock_server()

    server.test(body={'inputs': 
                      pd.DataFrame(json.loads(project2.get_artifact('log-transactions-log_transactions_encoded_iris-v3').to_dataitem().get())).values.tolist()[-5:]})
except Exception as e:
    print(e)

command file http://cross-project1-admin-proj1-serving-cross-project1-admin.default-tenant.app.vmdev94.lab.iguazeng.com/ not found


In [15]:
# Redeploying the already deployed function
project2.deploy_function('proj2_imported_serving')

> 2023-01-11 13:03:40,484 [info] Starting remote function deploy
2023-01-11 13:03:40  (info) Deploying function
2023-01-11 13:03:40  (info) Building
2023-01-11 13:03:40  (info) Staging files and preparing base images
2023-01-11 13:03:40  (info) Building processor image
2023-01-11 13:06:06  (info) Build complete
2023-01-11 13:06:34  (info) Function deploy complete
> 2023-01-11 13:06:34,819 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-cross-project2-admin-proj1-serving.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['cross-project2-admin-proj1-serving-cross-project2-admin.default-tenant.app.vmdev94.lab.iguazeng.com/']}


DeployStatus(state=ready, outputs={'endpoint': 'http://cross-project2-admin-proj1-serving-cross-project2-admin.default-tenant.app.vmdev94.lab.iguazeng.com/', 'name': 'cross-project2-admin-proj1-serving'})

##### Testing the redeployed imported-deployed-function

In [16]:
time.sleep(5) 
project2.get_function('proj2_imported_serving').invoke(
    path='/v2/models/my_model/infer', 
    body={'inputs': pd.DataFrame(json.loads(project2.get_artifact('log-transactions-log_transactions_encoded_iris-v3').to_dataitem().get())).values.tolist()[-5:]}
)

> 2023-01-11 13:06:39,974 [info] invoking function: {'method': 'POST', 'path': 'http://nuclio-cross-project2-admin-proj1-serving.default-tenant.svc.cluster.local:8080/v2/models/my_model/infer'}


{'id': '436ab3d0-c6f4-43a4-90c1-e4320378078b',
 'model_name': 'my_model',
 'outputs': [2, 2, 2, 2, 2]}

#### 4.1.3 Importing & Exporting undeployed serving function

In [17]:
# Setting project2 serving function
project2.set_function(mlrun.new_function(name='proj2_serving', kind='serving',image='mlrun/ml-models', command = []))

# adding project1 model to project2 serving function
project2.get_function('proj2-serving').add_model(key = 'my_model',
                                                 class_name = "mlrun.frameworks.xgboost.XGBoostModelServer",
                                                 model_path = project1.get_artifact('log-transactions-log_transactions_bst_model').target_path)

In [18]:
# exporting project2 not deployed serving function
project2.get_function('proj2-serving').export('proj2_serving.yaml')

# Importing project2 serving function from project1
project1.set_function(name='proj1_imported_serving', func='../proj2_serving.yaml')
print(project1.get_function('proj1_imported_serving').is_deployed())

> 2023-01-11 13:06:40,271 [info] function spec saved to path: proj2_serving.yaml
True


In [19]:
# Testing the imported undeployed function
server = project1.get_function('proj1_imported_serving').to_mock_server()

server.test(body={'inputs': 
                  pd.DataFrame(json.loads(project2.get_artifact('log-transactions-log_transactions_encoded_iris-v3').to_dataitem().get())).values.tolist()[-5:]})

> 2023-01-11 13:06:40,291 [warning] run command, file or code were not specified
> 2023-01-11 13:06:41,205 [info] model my_model was loaded
> 2023-01-11 13:06:41,206 [info] Loaded ['my_model']


{'id': '8bd932adc96d43369b357a8d5685b997',
 'model_name': 'my_model',
 'outputs': [2, 2, 2, 2, 2]}

In [20]:
# Deploying the undeployed imported function
project1.deploy_function('proj1_imported_serving')

> 2023-01-11 13:06:41,300 [info] Starting remote function deploy
2023-01-11 13:06:41  (info) Deploying function
2023-01-11 13:06:41  (info) Building
2023-01-11 13:06:41  (info) Staging files and preparing base images
2023-01-11 13:06:41  (info) Building processor image
2023-01-11 13:09:05  (info) Build complete
2023-01-11 13:09:33  (info) Function deploy complete
> 2023-01-11 13:09:34,470 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-cross-project1-admin-proj2-serving.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['cross-project1-admin-proj2-serving-cross-project1-admin.default-tenant.app.vmdev94.lab.iguazeng.com/']}


DeployStatus(state=ready, outputs={'endpoint': 'http://cross-project1-admin-proj2-serving-cross-project1-admin.default-tenant.app.vmdev94.lab.iguazeng.com/', 'name': 'cross-project1-admin-proj2-serving'})

In [21]:
time.sleep(5)

# Testing the imported then deployed function
project1.get_function('proj1_imported_serving').invoke(
    path='/v2/models/my_model/infer', 
    body={'inputs': pd.DataFrame(json.loads(project2.get_artifact('log-transactions-log_transactions_encoded_iris-v3').to_dataitem().get())).values.tolist()[-5:]}
)

> 2023-01-11 13:09:39,613 [info] invoking function: {'method': 'POST', 'path': 'http://nuclio-cross-project1-admin-proj2-serving.default-tenant.svc.cluster.local:8080/v2/models/my_model/infer'}


{'id': '698bec70-6896-4a3a-8278-48e237eb5183',
 'model_name': 'my_model',
 'outputs': [2, 2, 2, 2, 2]}

### 4.2 mlrun function

In [22]:
# exporting project2 log_transaction mlrun function
project2.get_function('log_transactions').export('proj2_mlrun_func.yaml')

# Importing project2 log_transaction mlrun function from project1
project1.set_function(name='proj1_imported_mlrun_func', func='../proj2_mlrun_func.yaml')
project1.run_function('proj1_imported_mlrun_func')

> 2023-01-11 13:09:39,937 [info] function spec saved to path: proj2_mlrun_func.yaml
> 2023-01-11 13:09:40,162 [info] starting run log-transactions-log_transactions uid=8d52e1e5a28b4d5c902cd496174447fc DB=http://mlrun-api:8080
> 2023-01-11 13:09:40,404 [info] Job is running in the background, pod: log-transactions-log-transactions-zkgnq
> 2023-01-11 13:09:45,610 [info] dataframe shape : (150, 4)
> 2023-01-11 13:09:45,717 [info] logged model : <class 'xgboost.sklearn.XGBClassifier'>
> 2023-01-11 13:09:45,868 [info] logged dataset (150, 5)
> 2023-01-11 13:09:45,914 [info] To track results use the CLI: {'info_cmd': 'mlrun get run 8d52e1e5a28b4d5c902cd496174447fc -p cross-project1-admin', 'logs_cmd': 'mlrun logs 8d52e1e5a28b4d5c902cd496174447fc -p cross-project1-admin'}
> 2023-01-11 13:09:45,914 [info] Or click for UI: {'ui_url': 'https://dashboard.default-tenant.app.vmdev94.lab.iguazeng.com/mlprojects/cross-project1-admin/jobs/monitor/8d52e1e5a28b4d5c902cd496174447fc/overview'}
> 2023-01-1

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
cross-project1-admin,...174447fc,0,Jan 11 13:09:45,completed,log-transactions-log_transactions,v3io_user=adminkind=jobowner=adminmlrun/client_version=1.3.0-rc4host=log-transactions-log-transactions-zkgnq,,,,encoded_iris-v3bst_modelmy_projectiris_dataset-v3


> 2023-01-11 13:09:46,875 [info] run executed, status=completed


### 4.3 nuclio function

#### 4.3.1 Importing & Exporting deployed remote function

In [23]:
%%writefile script.py
import mlrun
import time

def handler(context: mlrun.MLClientCtx, event):
    context.logger.info('Going to sleep zZz...')
    time.sleep(5)
    return 'Waking up !'

Writing script.py


In [24]:
# Setting remote function
project1.set_function(func = mlrun.code_to_function(name='proj1_remote_func', kind='remote',image='mlrun/mlrun', filename='script.py', handler='handler'))

# Deploying
project1.deploy_function('proj1-remote-func')
# Exporting project1 undeployed remote function
project1.get_function('proj1-remote-func').export('proj1_remote_func.yaml')

# Importing project2 remote function from project1
project2.set_function(name='proj2_imported_remote_func', func='../proj1_remote_func.yaml')

> 2023-01-11 13:09:47,001 [info] Starting remote function deploy
2023-01-11 13:09:47  (info) Deploying function
2023-01-11 13:09:47  (info) Building
2023-01-11 13:09:47  (info) Staging files and preparing base images
2023-01-11 13:09:47  (info) Building processor image
2023-01-11 13:10:52  (info) Build complete
2023-01-11 13:11:37  (info) Function deploy complete
> 2023-01-11 13:11:37,169 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-cross-project1-admin-proj1-remote-func.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['cross-project1-admin-proj1-remote-func-cross-project1-admin.default-tenant.app.vmdev94.lab.iguazeng.com/']}
> 2023-01-11 13:11:37,246 [info] function spec saved to path: proj1_remote_func.yaml


In [25]:
project2.deploy_function('proj2_imported_remote_func')

time.sleep(5)
project2.get_function('proj2_imported_remote_func').invoke('')

> 2023-01-11 13:11:37,278 [info] Starting remote function deploy
2023-01-11 13:11:37  (info) Deploying function
2023-01-11 13:11:37  (info) Building
2023-01-11 13:11:37  (info) Staging files and preparing base images
2023-01-11 13:11:37  (info) Building processor image
2023-01-11 13:12:45  (info) Build complete
> 2023-01-11 13:13:33,667 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-cross-project2-admin-proj1-remote-func.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['cross-project2-admin-proj1-remote-func-cross-project2-admin.default-tenant.app.vmdev94.lab.iguazeng.com/']}
> 2023-01-11 13:13:38,725 [info] invoking function: {'method': 'GET', 'path': 'http://nuclio-cross-project2-admin-proj1-remote-func.default-tenant.svc.cluster.local:8080/'}


b'Waking up !'

## 5. Importing/Exporting project

In [26]:
# Initialize the MLRun project object
new_project = mlrun.get_or_create_project('testing-exported-proj', context=os.path.join(os.getcwd(), 'new-project'))

# Required credentials :
# AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, GOOGLE_APPLICATION_CREDENTIALS, S3_BUCKET
AWS_ACCESS_KEY_ID = os.environ.get('AWS_ACCESS_KEY_ID', None)
AWS_SECRET_ACCESS_KEY = os.environ.get('AWS_SECRET_ACCESS_KEY', None)
GOOGLE_APPLICATION_CREDENTIALS = os.environ.get('GOOGLE_APPLICATION_CREDENTIALS', None)
assert AWS_ACCESS_KEY_ID != None and AWS_SECRET_ACCESS_KEY != None and GOOGLE_APPLICATION_CREDENTIALS != None

secrets = {'AWS_ACCESS_KEY_ID': AWS_ACCESS_KEY_ID,
           'AWS_SECRET_ACCESS_KEY':AWS_SECRET_ACCESS_KEY,
           'GOOGLE_APPLICATION_CREDENTIALS': GOOGLE_APPLICATION_CREDENTIALS}

new_project.set_secrets(secrets=secrets, provider='kubernetes')

S3_BUCKET = os.environ.get('S3_BUCKET', 'testbucket-igz')

new_project.artifact_path = os.path.join('s3://', S3_BUCKET + '/cross_project/')

> 2023-01-11 13:13:44,108 [info] loaded project testing-exported-proj from MLRun DB


In [27]:
# Setting artifacts
new_project.set_artifact('encoded_iris', artifact=os.path.join(os.getcwd(),'proj2_encoded_iris.yaml'))
new_project.set_artifact('model', artifact=os.path.join(os.getcwd(),'proj1_model.yaml'))
new_project.set_artifact('dataset', artifact=os.path.join(os.getcwd(),'proj2_dataset.yaml'))
new_project.set_artifact('dir', artifact=os.path.join(os.getcwd(),'proj1_dir.yaml'))

# Setting functions
new_project.set_function(name='serving', func='../proj1_serving.yaml')
new_project.set_function(name='mlrun_func', func='../proj2_mlrun_func.yaml')
new_project.set_function(name='remote_func', func='../proj1_remote_func.yaml')


In [28]:
# exporting the project

# S3 artifact path (e.g. s3://my-bucket/new_project.zip)
new_project.export(filepath=os.path.join(new_project.artifact_path, 'new_project.zip'))

# GCS artifact path (e.g. gs://my-bucket/new_project.zip)
new_project.export(filepath=os.path.join('gs' + new_project.artifact_path[2:], 'new_project.zip'))

# V3IO local path
new_project.export(filepath='/v3io/bigdata/new_project.zip')

In [29]:
# Importing the projects
gs_project = mlrun.load_project(name='testing-gs-project',
                                url=os.path.join(new_project.artifact_path, 'new_project.zip'),
                                context = os.path.join(os.getcwd(), 'gs-project'))

s3_project = mlrun.load_project(name='testing-s3-project',
                                url=os.path.join('s3' + new_project.artifact_path[2:], 'new_project.zip'),
                                context = os.path.join(os.getcwd(), 's3-project'))

v3io_project = mlrun.load_project(name='testing-v3io-project',
                                  url='/v3io/bigdata/new_project.zip',
                                  context=os.path.join(os.getcwd(), 'v3io-project'))

In [30]:
# Testing the imported artifacts
# Artifact
gs_encoded_dataset = pd.DataFrame(json.loads(gs_project.get_artifact('encoded_iris').to_dataitem().get()))
print('gs project artifact\n', gs_encoded_dataset.head(), '\n\n')

# Dataset
s3_dataset = s3_project.get_artifact('dataset').to_dataitem().as_df()
print('s3 project artifact\n',s3_dataset.head())

# Model
v3io_model = pickle.loads(v3io_project.get_artifact('model')._get_file_body())
v3io_model.predict(gs_encoded_dataset[:5])

# Directory
gs_dir = gs_project.get_artifact('dir').to_dataitem().listdir()

# Testing imported functions
# Serving function
gs_project.get_function('serving').deploy()
time.sleep(5)
gs_project.get_function('serving').invoke(
    path='/v2/models/my_model/infer', 
    body={'inputs': gs_encoded_dataset.values[-5:].tolist()}
)

# mlrun function
s3_project.run_function('mlrun_func',local=True)

## nuclio function
v3io_project.deploy_function('remote_func')

time.sleep(5)
v3io_project.get_function('remote_func').invoke('')

gs project artifact
      0    1    2    3
0  5.1  3.5  1.4  0.2
1  4.9  3.0  1.4  0.2
2  4.7  3.2  1.3  0.2
3  4.6  3.1  1.5  0.2
4  5.0  3.6  1.4  0.2 


s3 project artifact
    Unnamed: 0    0    1    2    3
0           0  5.1  3.5  1.4  0.2
1           1  4.9  3.0  1.4  0.2
2           2  4.7  3.2  1.3  0.2
3           3  4.6  3.1  1.5  0.2
4           4  5.0  3.6  1.4  0.2
> 2023-01-11 13:14:39,279 [info] Starting remote function deploy
2023-01-11 13:14:39  (info) Deploying function
2023-01-11 13:14:39  (info) Building
2023-01-11 13:14:39  (info) Staging files and preparing base images
2023-01-11 13:14:39  (info) Building processor image
2023-01-11 13:16:57  (info) Build complete
2023-01-11 13:17:33  (info) Function deploy complete
> 2023-01-11 13:17:34,104 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-testing-gs-project-proj1-serving.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['testing-gs-project-proj1-serving-testing-gs-pr

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
testing-s3-project,...e722cf01,0,Jan 11 13:17:39,completed,log-transactions-log_transactions,v3io_user=adminkind=owner=adminhost=jupyter-6479d4cd8c-rnxrj,,,,encoded_iris-s3bst_modelmy_projectiris_dataset-s3


> 2023-01-11 13:17:50,568 [info] run executed, status=completed
> 2023-01-11 13:17:50,712 [info] Starting remote function deploy
2023-01-11 13:17:50  (info) Deploying function
2023-01-11 13:17:50  (info) Building
2023-01-11 13:17:50  (info) Staging files and preparing base images
2023-01-11 13:17:50  (info) Building processor image
2023-01-11 13:18:45  (info) Build complete
2023-01-11 13:19:33  (info) Function deploy complete
> 2023-01-11 13:19:34,303 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-testing-v3io-project-proj1-remote-func.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['testing-v3io-project-proj1-remote-func-testing-v3io-project.default-tenant.app.vmdev94.lab.iguazeng.com/']}
> 2023-01-11 13:19:39,361 [info] invoking function: {'method': 'GET', 'path': 'http://nuclio-testing-v3io-project-proj1-remote-func.default-tenant.svc.cluster.local:8080/'}


b'Waking up !'

## Cleanup

In [31]:
projects = [project1,project2,gs_project,s3_project,v3io_project, new_project]
for project in projects:
    mlrun.get_run_db().delete_project(name=project.name, deletion_strategy='cascade')

import shutil
for f in os.listdir():
    if (not f.endswith('ipynb')) and f != '.test':
        if os.path.isfile(f):
            os.remove(f)
        elif os.path.isdir(f):
            shutil.rmtree(f)
        else:
            raise "A file that is not a notebook wasn't deleted"

In [32]:
import boto3
from urllib.parse import urlparse

s3 = boto3.resource('s3')
bucket = s3.Bucket(urlparse(project.artifact_path).netloc)
bucket.objects.filter(Prefix=urlparse(project.artifact_path).path[1:]).delete()

[{'ResponseMetadata': {'RequestId': 'G3XBPP1C33AKNKM9',
   'HostId': 'K0wGC0ZHzfXGkvrmFW8V3oAa5MiBBl+NeRlmzPcwR9qg8RVDJO75VC6GDtRnRy+eUBRkNmurBQs=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'K0wGC0ZHzfXGkvrmFW8V3oAa5MiBBl+NeRlmzPcwR9qg8RVDJO75VC6GDtRnRy+eUBRkNmurBQs=',
    'x-amz-request-id': 'G3XBPP1C33AKNKM9',
    'date': 'Wed, 11 Jan 2023 13:20:20 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'cross_project/log-transactions-log_transactions/0/bst_model/model_spec.yaml'},
   {'Key': 'cross_project/log-transactions-log_transactions/0/encoded_iris-s3.csv'},
   {'Key': 'cross_project/new_project.zip'},
   {'Key': 'cross_project/log-transactions-log_transactions/0/bst_model/bst.pkl'},
   {'Key': 'cross_project/log-transactions-log_transactions/0/iris_dataset-s3.csv'}]}]